In [1]:
import mysql.connector
import json
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()
# Load database configuration from a JSON file
with open("config.json", "r") as f:
    config = json.load(f)

In [5]:
# Create a database connection
cnx = mysql.connector.connect(
    host="localhost",  # Update with your host
    user=config["username"],
    password=config["password"],
    database="project"
)
# Create a cursor object to execute SQL queries
cursor = cnx.cursor()


In [3]:
# Execute the first query and fetch the results
query1 = "SELECT Name, Location FROM Organization;"
cursor.execute(query1)
results1 = cursor.fetchall()
df=pd.DataFrame(results1,columns=["ORG","Location"])
print(df.head())

    ORG   Location
0  Org1  Location1
1  Org2  Location2
2  Org3  Location3
3  Org4  Location4
4  Org5  Location5


In [4]:
query2 = "SELECT CommentID,Content,UserID,PostID FROM Comment;"
cursor.execute(query2)
results2 = cursor.fetchall()
df=pd.DataFrame(results2,columns=["CommentID","Content","UserID","PostID"])
for i in range(len(df)):
    c_id=df["CommentID"][i]
    scores=sid.polarity_scores(df["Content"][i])
    print(f"{c_id}:{scores}")


1:{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
2:{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
3:{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
4:{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
5:{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
6:{'neg': 0.513, 'neu': 0.487, 'pos': 0.0, 'compound': -0.2755}


In [7]:
query2 = "SELECT * FROM User;"
cursor.execute(query2)
results2 = cursor.fetchall()
print(results2)

[(1, 'User1', 'user1@example.com', 'voter', 'password1', None), (2, 'User2', 'user2@example.com', 'voter', 'password2', None), (3, 'User3', 'user3@example.com', 'representative', 'password3', 1), (4, 'User4', 'user4@example.com', 'voter', 'password4', None), (5, 'User5', 'user5@example.com', 'representative', 'password5', 2)]


In [12]:
def get_user_by_email(email):
    query = f"SELECT * FROM User WHERE Email = '{email}'"
    cursor.execute(query)
    result = cursor.fetchall()
    return result

In [15]:
len(get_user_by_email("user1@example.com"))

1

In [16]:
len(get_user_by_email("atharva@example.com"))

0

In [23]:
user=get_user_by_email("user1@example.com")
user[0]

(1, 'User1', 'user1@example.com', 'voter', 'password1', None)

In [5]:
query="SELECT OrganizationID FROM Organization where Name='Org2'"
cursor.execute(query)
result = cursor.fetchall()
result[0][0]

2

In [6]:
query = f"SELECT * FROM User"
cursor.execute(query)
result = cursor.fetchall()
result

[(1, 'User1', 'user1@example.com', 'voter', 'password1', None),
 (2, 'User2', 'user2@example.com', 'voter', 'password2', None),
 (3, 'User3', 'user3@example.com', 'representative', 'password3', 1),
 (4, 'User4', 'user4@example.com', 'voter', 'password4', None),
 (5, 'User5', 'user5@example.com', 'representative', 'password5', 2),
 (6,
  'user1',
  'user1@gmail.com',
  'voter',
  '$2b$12$hxFBzK0HOFGyA8jGWl2VhOeb7aItlggC/Kz7rrhNyUPSP0Jq2ZRCm',
  3),
 (7,
  'atharvaam21',
  'atharvaam21@outlook.com',
  'voter',
  '$2b$12$OOrU1x9Jr8Cttgl9z0YBO.xZz/wBa7dM8i7SWPFel24/k0.OABTF2',
  3),
 (8,
  'amaan',
  'amaan@coep.com',
  'representative',
  '$2b$12$91Wg8YtDJqRtxptOidZsCeqbJLCNg/WjOi.HUu6Fqz2X4SWB6VV3G',
  3),
 (9,
  'thakre',
  'thakre@thakre.com',
  'voter',
  '$2b$12$KemNCt9VRvr.vw6M.FMyDOgFJUUJDdyqkbcxU.pCXhn46Hm4GNmy6',
  3)]

In [7]:
query = f"SELECT * FROM Post"
cursor.execute(query)
result = cursor.fetchall()
result

[(1,
  'Post1',
  'Description1',
  datetime.datetime(2023, 11, 2, 9, 26, 24),
  'in progress',
  'environment',
  1,
  10,
  2,
  None),
 (2,
  'Post2',
  'Description2',
  datetime.datetime(2023, 11, 2, 9, 26, 24),
  'in progress',
  'social',
  2,
  15,
  3,
  None),
 (3,
  'Post3',
  'Description3',
  datetime.datetime(2023, 11, 2, 9, 26, 24),
  'in progress',
  'environment',
  3,
  5,
  1,
  1),
 (4,
  'Post4',
  'Description4',
  datetime.datetime(2023, 11, 2, 9, 26, 24),
  'in progress',
  'governance',
  4,
  8,
  0,
  None),
 (5,
  'Post5',
  'Description5',
  datetime.datetime(2023, 11, 2, 9, 26, 24),
  'in progress',
  'environment',
  5,
  12,
  4,
  2),
 (6,
  'tHIS is a Title (i am drunk)',
  'Description1',
  datetime.datetime(2023, 11, 2, 9, 31, 35),
  'in progress',
  'environment',
  3,
  10,
  2,
  None)]

In [23]:
def get_posts():
    query = f"SELECT Post.*,Name FROM Post join User using(UserID);"
    cursor.execute(query)
    result = cursor.fetchall()
    posts_df=pd.DataFrame(result,columns=["PostID","Title","Description","CreatedAtDate","Status","Topic","UserID","Upvotes","Downvotes","OrganizationID","PostedBy"])
    return posts_df

In [24]:
get_posts()

,PostID,Title,Description,CreatedAtDate,Status,Topic,UserID,Upvotes,Downvotes,OrganizationID,PostedBy
0,1,Post1,Description1,2023-11-02 09:26:24,in progress,environment,1,10,2,NaN,User1
1,2,Post2,Description2,2023-11-02 09:26:24,in progress,social,2,15,3,NaN,User2
2,3,Post3,Description3,2023-11-02 09:26:24,in progress,environment,3,5,1,1.0,User3
3,4,Post4,Description4,2023-11-02 09:26:24,in progress,governance,4,8,0,NaN,User4
4,5,Post5,Description5,2023-11-02 09:26:24,in progress,environment,5,12,4,2.0,User5
5,6,tHIS is a Title (i am drunk),Description1,2023-11-02 09:31:35,in progress,environment,3,10,2,NaN,User3
